# Data Cleaning

Credit modelling is an important part of determining risk when offering loans to potential borrowers.

In this notebook the Lending Club dataset is cleaned to prepare it for the machine learning model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# specify data type for id column to prevent DTypeWarning
mixed_col_dtype = {'id': 'object'}

loans_2007 = pd.read_csv('loans_2007.csv', dtype=mixed_col_dtype)
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
loans_2007.shape

(42538, 52)

The dataset contains many columns that need to be cut down on. Consulting the data dictionary identifies several columns which can be dropped containing redundant data or leaked information from the future. Cols will be dropped in stages.

The first set of cols to be dropped are:
- *id* - generated by Lending CLub for unique identification processes only
- *member_id* - same as above
- *funded_amnt* - leaks data from the future (after the loan has already been funded)
- *funded-amnt-inv* - same as above
- *grade* - contains redundant information as the interest rate col (*int_rate*)
- *sub_grade* - same as above
- *emp_title* - required other data and a lot of processing to be potentially useful
- *issue_d* - leaks data from the future

In [4]:
loans_2007.drop(['id',
                 'member_id',
                 'funded_amnt',
                 'funded_amnt_inv',
                 'grade',
                 'sub_grade',
                 'emp_title',
                 'issue_d'], axis=1, inplace=True)

The next set of cols to be dropped are:

- *zip_code* - redundant with *addr_state* col
- *out_prncp* - leaks data from the future
- *out_prncp_inv* - same as above
- *total_pymnt* - same as above
- *total_pymnt_inv* - same as above
- *total_rec_prncp* - same as above

In [5]:
loans_2007.drop(['zip_code',
                 'out_prncp',
                 'out_prncp_inv', 
                 'total_pymnt',
                 'total_pymnt_inv',
                 'total_rec_prncp'], axis=1, inplace=True)

Next set of cols to be dropped:

- *total_rec_int* - leaks data from the future
- *total_rec_late_fee* - same as above
- *recoveris* - same as above
- *collection_recovery_fee* - same as above
- *last_pymnt_d* - same as above
- *last_pymnt_amnt* - same as above

In [6]:
loans_2007.drop(['total_rec_int',
                 'total_rec_late_fee',
                 'recoveries',
                 'collection_recovery_fee',
                 'last_pymnt_d',
                 'last_pymnt_amnt'], axis=1, inplace=True)

In [7]:
loans_2007.shape

(42538, 32)

The data has been reduced from 52 to 32 cols.

The target col for the machine learning model was then selected to be the *loan_status* col. Currently this col contains text values which we need to encode.

In [8]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

There are 8 potential loan statuses. Non-obvious statuses are explained below:

- *Charged Off* - No longer a reasonable expectation of further payments.
- *Does not meet the credit policy. Status:Fully Paid* - Loan was paid off, but today the loan application would no longer meet the credit policy and wouldn't be approved.
- *Does not meet the credit policy. Status:Charged Off* - Same as above but for charged off loans.
- *In Grace Period* - Loan is past due date but still in grace period of 15 days.

From an investor's perspective, only the *fully paid* and *charged off* statuses are important.

The problem can then be treated as a binary classification problem.

The data will therefore be filtered for *fully paid* and *charged off*.

- *fully paid* = 1
- *charged off* = 0

In [9]:
# filter the data for fully paid and charged off loans
loans_2007 = loans_2007[(loans_2007['loan_status'] == 'Fully Paid') |
                        (loans_2007['loan_status'] == 'Charged Off')].copy().reset_index().drop('index',axis=1)

In [10]:
# classification mapping dictionary
mapping = {'Fully Paid': 1,
           'Charged Off': 0}

# map classification labels to loans
loans_2007.replace(mapping, inplace=True)

Some of the columns have only one unique value. These columns will not add any benefit to the machine learning model and should be dropped.

In [11]:
# list of dropped cols
drop_columns = []

# loop through cols and drop those with only 1 unique value
for col in loans_2007.columns:
    uniques = loans_2007[col].dropna().unique()
    if len(uniques) == 1:
        drop_columns.append(col)
        
# drop relevant cols
loans_2007.drop(drop_columns, axis=1, inplace=True)

# print dropped cols
drop_columns

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [12]:
# write clean csv to file
loans_2007.to_csv('clean_loans_2007.csv')